This notebook uses real diffraction pattern intensity data and creates `Intensity-Time` plots for individual lattice planes.

In [ ]:
import matplotlib.pyplot as plt
import pyFAI.gui.jupyter
import pyFAI
import fabio
from pyFAI.test.utilstest import UtilsTest
from pyFAI.calibrant import CALIBRANT_FACTORY
from pyFAI.goniometer import SingleGeometry
import math
import numpy as np
import pathlib
from tqdm import tqdm
import warnings
import yaml
import os
import re

import intensity_time_functions as analysis
%load_ext autoreload
%autoreload 2

figure_size = 20

## Load YAML file

The file paths and user inputs for the analysis of the Diamond 2022 experiment is included in the `yaml` configuration file, to record the inputs of the analysis.

The information in the `yaml` file can be accessed like this.

In [ ]:
# to load the example analysis
config_path = "yaml/config_EXAMPLE.yaml"

# to load the Diamond 2022 analysis
# config_path = "yaml/config_diamond_2022.yaml"
config = analysis.get_config(config_path)

## Define input and output file paths

Define input and output file paths for the intensity-time analysis.

In [ ]:
input_calibration_path = config["file_paths"]["input_calibration_path"]
print("The input path for the .poni calibration file is:", input_calibration_path)

input_path = config["file_paths"]["input_path"]
print("The input path for the experiment is:", input_path)

input_experiment_list = config["file_paths"]["input_experiment_list"]
print("The list of input experiment file paths to choose from is:", input_experiment_list)

output_path = config["file_paths"]["output_path"]
print("The output path for the experiment is:", output_path)

output_experiment_list = config["file_paths"]["output_experiment_list"]
print("The list of output experiment file paths to choose from is:", output_experiment_list)

The `order_number` defined below will set the experiment to analyse, choose from the list in the yaml file, starting at zero.

In [ ]:
order_number = 0

## Load calibration

Load a .poni calibration file, containing the calibration parameters from refinement to a calibrant diffraction pattern.

To create a .poni calibration file, please follow the procedure in the [pyFAI-integration-caking](https://github.com/LightForm-group/pyFAI-integration-caking) notebooks.

In [ ]:
ai = pyFAI.load(input_calibration_path)

## Plot a single slice

Plot a single slice image from the first image in the sequence. 

Use the numbers below to define two-theta bounds that only just contain a single diffraction pattern ring.

You can also alter the number of points and number of cakes to define the two-theta and azimuthal resolution. However, the values of 10000 and 360 seem to work best.

In [ ]:
input_filepath = input_path + input_experiment_list[order_number]
two_theta_min = 4.76
two_theta_max = 4.92
number_of_points = 10000 # 10000 gives best two-theta resolution, but 1000 is faster to run
number_of_cakes = 360 # 360 gives 1 degree azimuthal resolution

image_number = 0
v_max = 100
# v_max = 5000

analysis.plot_single_slice(ai, input_filepath, two_theta_min, two_theta_max, number_of_points, number_of_cakes, 
                           image_number, v_max)

## Define intensity-time plot inputs

Define user inputs for the intensity-time plot analysis.

In [ ]:
lattice_planes = config["user_inputs"]["lattice_planes"]
print("The hkil or hkl indices of the lattice plane peak labels is:", lattice_planes)

two_theta_min = config["user_inputs"]["two_theta_min"][order_number]
print("The list of two theta minimum bounds is:", two_theta_min)

two_theta_max = config["user_inputs"]["two_theta_max"][order_number]
print("The list of two theta maximum bounds is:", two_theta_max)

v_max = config["user_inputs"]["v_max"][order_number]
print("The colormap data range is:", v_max)

number_of_points = config["user_inputs"]["number_of_points"][order_number]
print("The number of points in two-theta for the integration is:", number_of_points)

number_of_cakes = config["user_inputs"]["number_of_cakes"][order_number]
print("The number of azimuthal cakes is:", number_of_cakes)

image_step = config["user_inputs"]["image_step"][order_number]
print("The image step to analyse the data is every :", image_step, " images.")

acquisition_frequency = config["user_inputs"]["acquisition_frequency"][order_number]
print("The acquisition frequency of the data collection (in Hz) is:", acquisition_frequency)

In [ ]:
input_filepath = input_path + input_experiment_list[order_number]
print("The input filepath for the experiment is:", input_filepath)

output_filepath = output_path + output_experiment_list[order_number]
print("The output filepath for the experiment is:", output_filepath)

## Calculate intensity changes over time for different lattice planes

Use the following function to save the intensity changes with time for a specific lattice plane.

In [ ]:
result_dict = analysis.get_intensity_time(ai, input_filepath, output_filepath, lattice_planes, 
                                          two_theta_min, two_theta_max, number_of_points, number_of_cakes, 
                                          image_step)

## Plot intensity changes over time for individual lattice planes

The result dictionary can be accessed using the lattice plane strings as the keys.

We can therefore plot the intensity changes for a single lattice plane with time. 

**Uncomment the lines in the cell below** to test different values for the v_max parameter for producing the best image.

In [ ]:
# lattice_plane_number = 5
# v_max = 20000

# analysis.plot_intensity_time(result_dict, lattice_plane_number, lattice_planes, v_max, 
#                              acquisition_frequency, image_step, output_filepath, test = True)

## Save intensity-time plots for different lattice planes

To plot out the final intensity-time plots for all of our lattice planes, we can use the following function.

*Note, be sure to reload the input parameters from the yaml text file to set the v_max parameters.*

In [ ]:
analysis.save_intensity_time(result_dict, lattice_planes, v_max, 
                             acquisition_frequency, image_step, output_filepath)

## Load text data for intensity-time plots

The intensity-time data that has been saved as text files can be reloaded for further analysis using the function below.

*Note, this uses the output filepath defined in the yaml text file, and used for saving the original data.*

In [ ]:
load_result_dict, load_lattice_planes = analysis.load_intensity_time_data(output_filepath)

We can then test different values for the v_max parameter by **uncommenting the lines in the cell below** 

*Note, using load_result_dict as a new parameter in the function.
But, still using lattice_planes as the old parameter, due to glob reordering.*

In [ ]:
# lattice_plane_number = 5
# v_max = 20000

# analysis.plot_intensity_time(load_result_dict, lattice_plane_number, lattice_planes, v_max, 
#                              acquisition_frequency, image_step, output_filepath, test = True)

And we can replot the final intensity-time plots for all of our lattice planes.

*Note, using load_result_dict as a new parameter in the function.
But, still using lattice_planes as the old parameter, due to glob reordering.*

In [ ]:
analysis.save_intensity_time(load_result_dict, lattice_planes, v_max, 
                             acquisition_frequency, image_step, output_filepath)